# CIA BENCHMARKING

## Description

In this notebook we compared **CIA** with **other classification tools** among the most valid and compatible with Scanpy workflow:
- **Celltypist** (*Domínguez et al., Science, 2022* [[1]](https://www.science.org/doi/10.1126/science.abl5197)) which, exploiting a **combined approach based on Logistic Regression and Stocastic Gradient Descent**, extracts the most revelevant feature of **each cluster of the training set** and **trains a classification model**. 
- **Besca** auto_annot module (*Mädler et al., NAR Genomics and Bioinformatics, 2021* [[2]](https://doi.org/10.1093/nargab/lqab102)), which can be run with **Logistic Regression** (LR) or **Support Vector Machine** (SVM) algorithms.

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import multiprocessing
from functools import partial
from scipy.sparse import issparse
from scipy import sparse
import time
from sklearn import metrics
from scipy import sparse
from cia import investigate, report, external
import pickle
import celltypist
import besca as bc

## Training dataset

We **trained all the classifiers** with *Hao et al., 2021* [[3]](https://www.sciencedirect.com/science/article/pii/S0092867421005833) **PBMC atlas**, the same from which we extracted the gene signatures in [CIA workflow tutorial](../workflow/Cluster_Independent_Annotation.ipynb). 

In [ ]:
# To download the dataset (2.536 GB) 
!wget https://datasets.cellxgene.cziscience.com/b0381820-6536-487a-85d2-b5994ae0f1c8.h5ad -O data/b0381820-6536-487a-85d2-b5994ae0f1c8.h5ad


In [ ]:
# to read the atlas data
adata= sc.read('data/b0381820-6536-487a-85d2-b5994ae0f1c8.h5ad')
# to start from a count matrix
adata.X = adata.layers['corrected_counts']
# to normalize the data
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
# to set AnnData.raw attribute
adata.raw= adata
# to remove 'other T' cells
adata=adata[adata.obs['celltype.l1']!='other T']
adata.obs['Cell type']=adata.obs['celltype.l1']
# to rename clusters
adata.obs['Cell type']=adata.obs['Cell type'].cat.rename_categories(['B', 'CD4 T', 'CD8 T', 'DC', 'Mono', 'NK', 'Platelet'])
adata.uns['Cell type_colors']=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
adata

For standardization purposes and because of **Celltypist strict requirements**, we rescaled counts from 0 to 10000 for each cell and we log transformed the resulting values, accordingly with [Scanpy tutorial](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html). 

In [ ]:
sc.set_figure_params(scanpy=True, dpi=50)

In [ ]:
sc.pl.umap(adata, color='Cell type')

## Test dataset

We **fitted the trained model of each classifier on** [PBMC3K](https://scanpy.readthedocs.io/en/stable/generated/scanpy.datasets.pbmc3k.html) from *Satija et al., 2015* [[4]](https://www.nature.com/articles/nbt.3192), and we compared the performances of classification with CIA ones. 

NB.: The test dataset was normalized in the very same way of the training dataset.

In [ ]:
pbmc3k=sc.read('data/pbmc3k_classified.h5ad')

## Cell level classification

Since, like CIA, **Celltypist and Besca** allow to **automatically classify datasets cell-by-cell**, without requiring the clustering step, we challanged both classifiers at single cell level.

### Celltypist

In [ ]:
# To train Celltypist with PBMC atlas feature and ground truth labels ('Cell type' obs).

# COMPUTATIONALLY HEAVY! results have been stored in 'data/model_atlas.pkl'.
#new_model = celltypist.train(adata.raw.to_adata(), labels = 'Cell type', n_jobs = 32, feature_selection = True)

# To load trained model
new_model=celltypist.Model.load('data/model_atlas_no_other_t.pkl')
new_model

In [ ]:
# To fit the model and predict pbmc3k cell identities.
predictions = celltypist.annotate(pbmc3k.raw.to_adata(), model = new_model)
end=time.time()

In [ ]:
pbmc3k.obs['Prediction celltypist']=predictions.predicted_labels

In [ ]:
sc.pl.umap(pbmc3k, color=['Cell type', 'Prediction fast mode','Prediction p-val','Prediction celltypist'], wspace=0.5)

### Besca

In [ ]:
# to train Besca Logistic Regression model and perform feature selection
adata_train, adata_test_corrected = bc.tl.auto_annot.merge_data([adata], pbmc3k, genes_to_use = 'all', merge = 'scanorama')

In [ ]:
classifier, scaler = bc.tl.auto_annot.fit(adata_train, method='logistic_regression', celltype='Cell type',celltype_variable='Cell type', njobs=32)
bc.tl.auto_annot.adata_predict(classifier = classifier, adata_pred = adata_test_corrected, adata_orig=pbmc3k,
                               threshold =0.1, scaler=scaler)
pbmc3k.obs['Prediction besca LR']=pbmc3k.obs['auto_annot']
del pbmc3k.obs['auto_annot']

In [ ]:
# to train Besca SVM model and perform feature selection

# COMPUTATIONALLY HEAVY, results of this chunk have been already saved in pbmc3k.obs['auto_annot']

classifier, scaler = bc.tl.auto_annot.fit(adata_train, method='linear', celltype='Cell type', celltype_variable='Cell type', njobs=32)
bc.tl.auto_annot.adata_predict(classifier = classifier, adata_pred = adata_test_corrected, adata_orig=pbmc3k, 
                                threshold =0.1, scaler=scaler)
pbmc3k.obs['Prediction besca SVM']=pbmc3k.obs['auto_annot']
del pbmc3k.obs['auto_annot']

In [ ]:
sc.pl.umap(pbmc3k, color=['Cell type','Prediction fast mode','Prediction p-val', 'Prediction besca LR','Prediction besca SVM'], wspace=0.5)

### Performances

In [ ]:
report.classification_metrics(pbmc3k, classification_obs=['Prediction fast mode',
                                                         'Prediction p-val', 'Prediction celltypist',
                                                         'Prediction besca LR','Prediction besca SVM'], groups_obs='Cell type')

Notably, **CIA fast mode prediction was the best one**, followed by the computationally heavier Besca SVM prediction and CIA p-val mode (which are comparable).

## Over-clustering driven approach

**Celltypist** has an interesting feature called **'majority voting'**, which consists of the refinement of cell identities within local subclusters after an **over-clustering step**. Basically, **for each subcluster**, the **label of the most abundant cell type is extended** to the whole cell group.
This is an **additional step that goes beyond the CIA workflow**, but since we wanted to compare the results at the best possible conditions of classification, we wrote ***celltypist_majority_vote*** function (**external module**) to reproduce the 'majority voting' approach.

To be more clear, **each of the cell-level classifications were matched with those 68 clusters** (small groups of very similar cells) and at each mini-cluster was assigned the most abundant cell type label.

In [ ]:
sc.pl.umap(pbmc3k, color='leiden_5')

### Celltypist majority voting

In [ ]:
# To run Celltypist with majority voting
predictions = celltypist.annotate(pbmc3k.raw.to_adata(), model = new_model, majority_voting=True)

In [ ]:
pbmc3k.obs['Prediction celltypist majority voting']=predictions.predicted_labels['majority_voting']

In [ ]:
sc.pl.umap(pbmc3k, color=['Cell type','Prediction celltypist majority voting'], wspace=0.6)

### Majority voting embedded in CIA

In [ ]:
# To replicate the majority voting step on CIA annotations exploiting celltypist_majority_vote.
colnames=['Prediction fast mode', 'Prediction p-val', 'Prediction q','Prediction standard mode']
external.celltypist_majority_vote(pbmc3k,classification_obs=colnames)

In [ ]:
colnames_mv=['Cell type','Prediction fast mode majority voting', 'Prediction p-val majority voting',
            'Prediction q majority voting', 'Prediction standard mode majority voting']

In [ ]:
sc.pl.umap(pbmc3k, color=colnames_mv, wspace=0.4)

### Majority voting applied to Besca

In [ ]:
colnames=['Prediction besca LR','Prediction besca SVM']
external.celltypist_majority_vote(pbmc3k,classification_obs=colnames)

In [ ]:
colnames_mv=['Cell type','Prediction besca LR majority voting', 'Prediction besca SVM majority voting']

In [ ]:
pbmc3k.uns['Prediction besca SVM majority voting_colors']=['#fb9a99', '#b2df8a', '#1f78b4', '#d62728', '#6fadfd', '#E4D00A',
       '#FF5733', '#762a83']

In [ ]:
sc.pl.umap(pbmc3k, color=colnames_mv, wspace=0.6)

### Performances

In [ ]:
report.classification_metrics(pbmc3k, classification_obs=['Prediction celltypist majority voting', 'Prediction fast mode majority voting', 'Prediction p-val majority voting', 
             'Prediction besca LR majority voting','Prediction besca SVM majority voting'], groups_obs='Cell type')

Notably, **all the classifications** resulted to have a **very high F1-score**. However, **CIA performed the best classifications** also after the majoity voting step, followed by Besca SVM (comparable).

## Conclusions

In this notebook we challenged **some of the most valid classifiers compatible with Scanpy** in annotating the [PBMC3K dataset](https://scanpy.readthedocs.io/en/stable/generated/scanpy.datasets.pbmc3k.html) and we compared their perfomances with CIA ones. Here are summarized all the results:

- Overall the **classifiers allowing cell-level annotation were able to correctly predict the identity of the majority of cells**, indicating that it's possible to accurately annotate the datasets without the arbitrariety which characterizes the clustering-driven manual annotation. Among them, **CIA resulted to be the best at cell-level annotation**. 
- **The winning approach** of this challenge **is cell-level automatic annotation followed by** over-clustering and **majority voting**. With this approach, clustering with very high resolution is performed only after cell-level classification. **When clustering is used to refine** an already accurate automatic labelling (and so less influenced by analysts arbitrary decisions), instead of completely drive the annotation, it **results to be an effective way to integrate** the contributions of **marker genes expression with** the more general concept of **transcriptional similarity** in defining cell identity. Also with this approach **CIA obtained the best results**.

#### Clasiffication Performances

In [ ]:
columns=['Prediction fast mode','Prediction p-val','Prediction q', 'Prediction standard mode',
         'Prediction celltypist', 'Prediction besca LR','Prediction besca SVM']

In [ ]:
report.classification_metrics(pbmc3k, classification_obs=columns, groups_obs='Cell type')

#### Clasiffication Performances - Majority Voting

In [ ]:
columns_majority=[ 'Prediction fast mode majority voting','Prediction p-val majority voting',
                  'Prediction q majority voting', 'Prediction standard mode majority voting', 
                  'Prediction celltypist majority voting', 
                  'Prediction besca LR majority voting','Prediction besca SVM majority voting']

In [ ]:
report.classification_metrics(pbmc3k, classification_obs=columns_majority, groups_obs='Cell type')

In [ ]:
pbmc3k.write('data/pbmc3k_classified.h5ad')